# 12 - Hyperopt

- This is a Python library for optimizing discrete and continuous hyperparameters for XGBoost.
- This uses a probabilistic model to select the next set of hyperparameters to try.

- aside: first class functions in python:
    - python allows you to pass a function in as a parameter to another function or return a function as a result.

- this is a really useful chapter!

In [1]:
from setup import * #gets the real data

/media/ryan/5e52f95b-dbda-4605-bec6-10d513d58772/GitHubProjExt/effective-xgboost-notes/setup.py:32: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,42,44,54,56,64,81,83,85,87,105,107,109,121,123,125,148,150,157,172,174,192,194,210,218,219,221,223,246,247,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,302,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag = pd.read_csv(z.open(member_name))
/media/ryan/5e52f95b-dbda-4605-bec6-10d513d58772/GitHubProjExt/effective-xgboost-notes/setup.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  education=df_.Q4.replace({'Master’s degree': 18,
/media/ryan/5e52f95b-dbda-4605-bec6-10d513d58772/GitHubProjEx

In [4]:
import numpy as np
import numpy.random as rn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, dummy, preprocessing, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
import dtreeviz
import yellowbrick.model_selection as ms
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from typing import Any, Dict, Union
#also, must install graphviz for plotting to work with xgb

In [5]:
def hyperparameter_tuning(space: Dict[str, Union[float, int]],\
    X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame,\
    y_test: pd.Series, early_stopping_rounds: int=50, metric:callable=accuracy_score) -> Dict[str, Any]:
    int_vals = ['max_depth','reg_alpha']
    space={k: (int(val) if k in int_vals else val) for k, val in space.items()}
    space['early_stopping_rounds'] = early_stopping_rounds
    model = xgb.XGBClassifier(**space)
    evaluation = [(X_train, y_train), (X_test, y_test)]
    model.fit(X_train, y_train, eval_set=evaluation, verbose=False)
    pred = model.predict(X_test)
    score = metric(y_test, pred)
    return {'loss': -score, 'status': STATUS_OK, 'model':model}

In [6]:
options = {'max_depth': hp.quniform('max_depth', 1, 8, 1), #tree
    'min_child_weight': hp.loguniform('min_child_weight',-2,3),
    'subsample': hp.uniform('subsample', 0.5, 1), #stochastic
    'colsample_bytree': hp.uniform('colsample_bytree', .5, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda',1,10),
    'gamma': hp.loguniform('gamma', -10, 10), #regularization
    'learning_rate': hp.loguniform('learning_rate',-7,0), #boosting
    'random_state':42
}   

trials = Trials()

In [9]:
best = fmin(fn=lambda space: hyperparameter_tuning(space, X_train, y_train, X_test, y_test), space=options, \
    algo=tpe.suggest, max_evals=2_000, trials=trials, timeout=60*5 ) #it will stop, but still give an answer at the time!

  0%|          | 0/2000 [00:00<?, ?trial/s, best loss=?]

 47%|████▋     | 942/2000 [05:00<05:37,  3.14trial/s, best loss: -0.7756906077348066]


## Pretty cool, there is a little tracker that comes with it

In [15]:
best['max_depth'] = int(best['max_depth'])

In [19]:
#if trust output despite time cutoff
long_params = best

In [ ]:
# long_params = {'max_depth': 6, #tree
#     'min_child_weight': 0.66784,
#     'subsample': 0.9998, #stochastic
#     'colsample_bytree': 0.6875,
#     'reg_alpha': 3.29799,
#     'reg_lambda': 7.8509,
#     'gamma': 0.0693632, #regularization
#     'learning_rate': 0.2143921, #boosting
#     'random_state':42
}   


In [18]:
# now can train a model with hyperparameters
xg_ex = xgb.XGBClassifier(**long_params, early_stopping_rounds=50, n_estimators=500)
xg_ex.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_test, y_test)], verbose=100)

[0]	validation_0-logloss:0.58802	validation_1-logloss:0.59120


[66]	validation_0-logloss:0.39076	validation_1-logloss:0.51176


/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.9/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [02:23:24] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "subsammple" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8415280350422588, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.015707122362867733,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.6273184258434865,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=3.7048292728340466, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [ ]:
xg_ex.score

## 12.3 Defining Parameter Distributions